# [建立檢索輔助生成 (RAG) 應用程式](https://python.langchain.com/v0.2/docs/tutorials/rag/)
================================

大型語言模型 (LLM) 所啟用的最強大應用之一就是高級問答 (Q&A) 聊天機器人。這類應用程式能夠針對特定的來源資訊回答問題，並使用一種稱為檢索輔助生成 (Retrieval Augmented Generation, RAG) 的技術。

本教程將展示如何在文本資料來源上建立一個簡單的Q&A應用程式。在此過程中，我們會介紹典型的Q&A架構，並強調一些進階Q&A技術的額外資源。此外，我們也會看到LangSmith如何幫助我們追踪和理解應用程式的運作。隨著應用程式的複雜度增加，LangSmith將變得越來越有幫助。

如果你已經熟悉基本的檢索技術，你可能也會對這個[不同檢索技術的高階概覽](https://python.langchain.com/v0.2/docs/concepts/#retrieval)感興趣。

什麼是RAG？[​](https://python.langchain.com/v0.2/docs/tutorials/rag/#what-is-rag "直接鏈接到什麼是RAG？")
------------------------------------------------------------

RAG是一種用來增強LLM知識的技術，透過額外的資料來輔助生成。

LLM能夠對廣泛的主題進行推理，但其知識僅限於訓練時截至某個時間點的公開資料。如果你想建立能夠處理私有資料或是在模型截止日期之後引入的資料的AI應用程式，你需要透過特定資訊來增強模型的知識。將適當的資訊引入並插入到模型提示中的過程被稱為檢索輔助生成 (RAG)。

LangChain 提供了多個元件來幫助建立Q&A應用程式，更廣泛地說，還能幫助建立RAG應用程式。

注意：這裡我們專注於非結構化資料的Q&A。如果你對於在結構化資料上使用RAG感興趣，請查看我們的[SQL資料問答教程](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/)。

概念[​](https://python.langchain.com/v0.2/docs/tutorials/rag/#concepts "直接鏈接到概念")
------------------------------------------------

一個典型的RAG應用程式有兩個主要的組成部分：

- 索引：一個從來源引入資料並進行索引的管道。*這通常是離線執行的。*
- 檢索與生成：實際的RAG鏈條，這會在運行時接收使用者的查詢，從索引中檢索相關資料，然後將其傳遞給模型。

從原始資料到答案的最常見全序列如下：

### 索引[​](https://python.langchain.com/v0.2/docs/tutorials/rag/#indexing "直接鏈接到索引")

1.  載入：首先，我們需要載入資料。這可以使用[文件載入器](https://python.langchain.com/v0.2/docs/concepts/#document-loaders)來完成。
2.  分割：使用[文本分割器](https://python.langchain.com/v0.2/docs/concepts/#text-splitters)將大的`Documents`分割成較小的塊。這對於索引資料和將其傳遞給模型都非常有用，因為大的塊不僅更難檢索，而且也無法適應模型有限的上下文窗口。
3.  儲存：我們需要一個地方來儲存和索引這些分割，以便之後可以檢索到它們。這通常是使用[向量儲存](https://python.langchain.com/v0.2/docs/concepts/#vector-stores)和[嵌入模型](https://python.langchain.com/v0.2/docs/concepts/#embedding-models)來完成的。

![index_diagram](https://python.langchain.com/v0.2/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

### [Retrieval and generation](https://python.langchain.com/v0.2/docs/tutorials/rag/#retrieval-and-generation "Direct link to Retrieval and generation")

1.  檢索：根據使用者的輸入，使用[檢索器](https://python.langchain.com/v0.2/docs/concepts/#retrievers)從儲存中檢索相關的資料分割。
2.  生成：使用[聊天模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)或[LLM](https://python.langchain.com/v0.2/docs/concepts/#llms)，透過包含問題和檢索資料的提示生成答案。
![retrieval_diagram](https://python.langchain.com/v0.2/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

In [ ]:
# ! pip3 install langchain langchain_community langchain_chroma bs4 langchainhub

許多使用 LangChain 開發的應用程序都包含多個步驟，其中涉及多次 LLM 調用。隨著這些應用程序變得越來越複雜，能夠檢查鏈或代理內部發生的情況變得至關重要。而檢查這些過程的最佳方法就是使用 [LangSmith](https://smith.langchain.com/)。

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

## [預覽​](https://python.langchain.com/v0.2/docs/tutorials/rag/#preview "直接鏈接到預覽")
--------------------------------------------------------------------------------------------------

在本指南中，我們將建立一個應用程式，用來回答關於網站內容的問題。我們將使用的具體網站是Lilian Weng撰寫的[LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)部落格文章，這樣我們可以針對文章內容提出問題。

我們可以用大約20行代碼來建立一個簡單的索引管道和RAG鏈條來實現這個目標：

In [6]:
from langchain_ollama import ChatOllama
model_name = "llama3.1"
llm = ChatOllama(model=model_name)
llm

ChatOllama(model='llama3.1')

In [7]:
from langchain_ollama import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    model=model_name,
)

In [9]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=ollama_emb)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


'Task Decomposition refers to breaking down complex tasks into smaller, manageable sub-tasks that can be solved using multiple API calls. This process involves identifying the necessary APIs, learning how to use them, and combining their outputs to achieve a specific goal. It is assessed at Level-3 of this benchmark.'

In [11]:
# cleanup
vectorstore.delete_collection()

檢視 [LangSmith trace](https://smith.langchain.com/public/1c6ca97e-445b-4d00-84b4-c7befcbc59fe/r).

### [詳細操作說明](https://python.langchain.com/v0.2/docs/tutorials/rag/#detailed-walkthrough "直接鏈接到詳細操作說明")
-----------------------------------------------------------------------------------------------------------------------------------------

現在，我們來一步步解析上面的代碼，以深入了解其運作原理。

1\. 索引：載入[​](https://python.langchain.com/v0.2/docs/tutorials/rag/#indexing-load "直接鏈接到1. 索引：載入")
-----------------------------------------------------------------------------------------------------------------------------

我們首先需要載入部落格文章的內容。這可以使用[文件載入器](https://python.langchain.com/v0.2/docs/concepts/#document-loaders)來完成，這些是從資料來源載入資料並返回[文件](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)列表的物件。`Document` 是一個包含 `page_content`（字串類型）和 `metadata`（字典類型）的物件。

在這個案例中，我們將使用[WebBaseLoader](https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/)，該載入器使用 `urllib` 來從網頁URL載入HTML，並使用 `BeautifulSoup` 解析HTML為文本。我們可以通過 `bs_kwargs` 傳遞參數給 `BeautifulSoup` 解析器來自定義HTML到文本的解析過程（參見[BeautifulSoup文檔](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)）。在這個案例中，只有HTML標籤中類別為 "post-content"、"post-title" 或 "post-header" 的內容是相關的，因此我們會移除其他所有的標籤。

In [13]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

In [12]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### [更深入探索​](https://python.langchain.com/v0.2/docs/tutorials/rag/#go-deeper "直接鏈接到更深入探索")

`DocumentLoader`：一個將資料從來源載入為 `Documents` 列表的物件。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/#document-loaders)：關於如何使用 `DocumentLoaders` 的詳細文件。
-   [整合](https://python.langchain.com/v0.2/docs/integrations/document_loaders/)：超過160個可供選擇的整合工具。
-   [介面](https://api.python.langchain.com/en/latest/document_loaders/langchain_core.document_loaders.base.BaseLoader.html)：基本介面的API參考。

2\. [索引：分割](https://python.langchain.com/v0.2/docs/tutorials/rag/#indexing-split "直接鏈接到2. 索引：分割")
--------------------------------------------------------------------------------------------------------------------------------

我們載入的文件長度超過42,000個字元，這太長了，無法適應許多模型的上下文窗口。即使是那些能夠在上下文窗口中容納完整文章的模型，面對如此長的輸入資料也會很難找出有用的資訊。

為了解決這個問題，我們將把 `Document` 分割成多個塊，便於嵌入和向量儲存。這將幫助我們在運行時檢索到部落格文章中最相關的部分。

在這個案例中，我們將文件分割成每個塊1000個字元，並且在Chunks之間有200個字元的重疊。這樣的重疊有助於減少斷開與重要上下文相關的陳述的可能性。我們使用 [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)，它會遞歸地使用常見的分隔符（如換行符）來分割文件，直到每個塊達到適當的大小。這是針對通用文本用例推薦的文本分割器。

我們設置 `add_start_index=True`，以便每個分割後的 `Document` 在初始文件中開始的字元索引保留為元數據屬性 "start_index"。

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

In [16]:
len(all_splits[0].page_content)

969

In [17]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

### [更深入探索​](https://python.langchain.com/v0.2/docs/tutorials/rag/#go-deeper-1 "直接鏈接到更深入探索")

`TextSplitter`：將 `Document` 列表分割成較小塊的物件，為 `DocumentTransformer` 的子類別。

-   透過閱讀[如何使用文檔](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)來了解使用不同方法分割文本的更多資訊。
-   [代碼 (py 或 js)](https://python.langchain.com/v0.2/docs/integrations/document_loaders/source_code/)
-   [學術論文](https://python.langchain.com/v0.2/docs/integrations/document_loaders/grobid/)
-   [介面](https://api.python.langchain.com/en/latest/base/langchain_text_splitters.base.TextSplitter.html)：基本介面的API參考。

`DocumentTransformer`：對 `Document` 物件列表進行轉換的物件。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)：關於如何使用 `DocumentTransformers` 的詳細文件。
-   [整合](https://python.langchain.com/v0.2/docs/integrations/document_transformers/)
-   [介面](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html)：基本介面的API參考。

3\. [索引：儲存​](https://python.langchain.com/v0.2/docs/tutorials/rag/#indexing-store "直接鏈接到3. 索引：儲存")
--------------------------------------------------------------------------------------------------------------------------------

現在我們需要對66個文本塊進行索引，以便在運行時能夠檢索它們。最常見的方法是將每個文件分割的內容嵌入並將這些嵌入插入向量資料庫（或向量儲存庫）中。當我們需要檢索這些分割的內容時，我們會將查詢文本嵌入，然後進行某種“相似性”搜索，以識別與查詢嵌入最相似的儲存分割。最簡單的相似性度量是餘弦相似度——我們測量每對嵌入之間的角度餘弦（這些嵌入是高維向量）。

我們可以使用 [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) 向量儲存庫和 [OpenAIEmbeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/) 模型，在一個命令中嵌入並儲存所有的文件分割。

In [18]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=ollama_emb)

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = da

### [更深入探索​](https://python.langchain.com/v0.2/docs/tutorials/rag/#go-deeper-2 "直接鏈接到更深入探索")

`Embeddings`：文本嵌入模型的包裝，用於將文本轉換為嵌入。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/embed_text/)：關於如何使用嵌入的詳細文檔。
-   [整合](https://python.langchain.com/v0.2/docs/integrations/text_embedding/)：超過30個可供選擇的整合工具。
-   [介面](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.Embeddings.html)：基本介面的API參考。

`VectorStore`：向量資料庫的包裝，用於儲存和查詢嵌入。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/vectorstores/)：關於如何使用向量儲存的詳細文檔。
-   [整合](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)：超過40個可供選擇的整合工具。
-   [介面](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html)：基本介面的API參考。

至此，我們完成了管道中索引的部分。現在，我們擁有一個可以查詢的向量儲存庫，其中包含我們部落格文章的分割內容。當有使用者提出問題時，我們應該能夠理想地返回該文章中回答問題的片段。

4\. [檢索與生成：檢索](https://python.langchain.com/v0.2/docs/tutorials/rag/#retrieval-and-generation-retrieve "直接鏈接到4. 檢索與生成：檢索")
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

現在我們來編寫實際的應用邏輯。我們想要建立一個簡單的應用程式，它可以接收使用者的問題，搜索與該問題相關的文件，將檢索到的文件和初始問題傳遞給模型，並返回答案。

首先，我們需要定義搜索文件的邏輯。LangChain 定義了一個[檢索器 (Retriever)](https://python.langchain.com/v0.2/docs/concepts/#retrievers/)介面，它封裝了一個索引，該索引可以根據字串查詢返回相關的 `Documents`。

最常見的 `Retriever` 類型是 [VectorStoreRetriever](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/)，它利用向量儲存的相似性搜索功能來促進檢索。任何 `VectorStore` 都可以通過 `VectorStore.as_retriever()` 輕鬆轉換為 `Retriever`：

In [19]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


6

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [20]:
print(retrieved_docs[0].page_content)

Fig. 4. Experiments on AlfWorld Env and HotpotQA. Hallucination is a more common failure than inefficient planning in AlfWorld. (Image source: Shinn & Labash, 2023)


### [更深入探索​](https://python.langchain.com/v0.2/docs/tutorials/rag/#go-deeper-3 "直接鏈接到更深入探索")

向量儲存通常用於檢索，但也有其他方式可以進行檢索。

`Retriever`：根據文本查詢返回 `Documents` 的物件

-   [文件](https://python.langchain.com/v0.2/docs/how_to/#retrievers)：有關此介面及內建檢索技術的更多文檔。其中一些包括：
    -   `MultiQueryRetriever` [生成輸入問題的變體](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/)，以提高檢索命中率。
    -   `MultiVectorRetriever` 生成[嵌入的變體](https://python.langchain.com/v0.2/docs/how_to/multi_vector/)，同樣是為了提高檢索命中率。
    -   `最大邊際相關性` 選擇檢索文件中的[相關性和多樣性](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf)，以避免傳遞重複的上下文。
    -   在向量儲存檢索過程中可以使用元數據過濾器對文件進行過濾，例如使用[自查詢檢索器 (Self Query Retriever)](https://python.langchain.com/v0.2/docs/how_to/self_query/)。
-   [整合](https://python.langchain.com/v0.2/docs/integrations/retrievers/)：與檢索服務的整合。
-   [介面](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html)：基本介面的API參考。

5\. [檢索與生成：生成​](https://python.langchain.com/v0.2/docs/tutorials/rag/#retrieval-and-generation-generate "直接鏈接到5. 檢索與生成：生成")
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

現在，我們將這一切結合在一起，構建一個鏈條，它會接收一個問題，檢索相關的文件，構建提示，將其傳遞給模型，並解析輸出。

我們將使用gpt-4o-mini OpenAI聊天模型，但也可以替換為任何LangChain的 `LLM` 或 `ChatModel`。

In [ ]:
from langchain_ollama import ChatOllama
model_name = "llama3.1"
llm = ChatOllama(model=model_name)
llm

ChatOllama(model='llama3.1')

我們將使用 RAG 的提示範本，此範本已經提交到 LangChain 提示範本中心 [這裡](https://smith.langchain.com/hub/rlm/rag-prompt)。

In [21]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [22]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


我們將使用 [LCEL Runnable](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) 協議來定義這個 Chain(鏈)，這樣可以讓我們：

- 透明地串聯組件和函數

- 自動在 LangSmith 中追蹤我們的鏈

- 開箱即用地獲得串流、非同步和批次呼叫功能

以下是實作內容：

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is a process that breaks down complex tasks into smaller and simpler steps, allowing an agent to plan ahead and manage the task more effectively. This is achieved through techniques like Chain of Thought (CoT), which instructs the model to "think step by step" and decompose hard tasks into manageable ones.

讓我們來剖析 LCEL，以了解其運作方式。

首先，這些組件（如 `retriever`、`prompt`、`llm` 等）都是 [Runnable](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) 的實例。這意味著它們實現了相同的方法，例如同步和非同步的 `.invoke`、`.stream` 或 `.batch`，使得它們更容易相互連接。這些實例可以透過 `|` 運算子連接成一個 [RunnableSequence](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html)，這也是另一個 Runnable。

當遇到 `|` 運算子時，LangChain 會自動將某些物件轉換為 runnables。在這裡，`format_docs` 會被轉換為 [RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html)，而包含 `"context"` 和 `"question"` 的字典會被轉換為 [RunnableParallel](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html)。細節不如更大的重點來得重要，即每個物件都是一個 Runnable。

現在讓我們來追蹤輸入問題如何通過上述這些 runnables。

如上所述，`prompt` 的輸入預期是一個包含 `"context"` 和 `"question"` 鍵的字典。因此，這條鏈的第一個元素會構建 runnables，從輸入問題中計算出這兩個鍵的值：

- `retriever | format_docs` 會將問題通過 retriever，生成 [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 對象，然後傳遞給 `format_docs` 以生成字符串；
- `RunnablePassthrough()` 會原封不動地傳遞輸入的問題。

也就是說，如果你構建了

In [24]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

然後，`chain.invoke(question)` 將會構建一個格式化的提示，準備進行推理。（注意：在使用 LCEL 進行開發時，可以通過這樣的子鏈來進行測試，這是很實用的。）

鏈的最後幾步是 `llm`，它負責執行推理，然後是 `StrOutputParser()`，這個步驟僅僅是從 LLM 的輸出消息中提取出字串內容。

你可以通過其 [LangSmith 追蹤](https://smith.langchain.com/public/1799e8db-8a6d-4eb2-84d5-46e8d7d5a99b/r) 來分析這個鏈的每個步驟。

### [Build-in-Chains 內建鏈](https://python.langchain.com/v0.2/docs/tutorials/rag/#built-in-chains "直接鏈接到內建鏈")

如果你需要，LangChain 提供了方便的函數來實現上述的 LCEL。我們將會組合兩個函數：

- [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) 指定如何將檢索到的上下文傳遞到提示和 LLM 中。在這個情況下，我們會將內容 "stuff" 到提示中——也就是說，我們會將所有檢索到的上下文全部包含在提示中，而不進行任何摘要或其他處理。它基本上實現了我們上面提到的 `rag_chain`，輸入鍵為 `context` 和 `input`，它使用檢索到的上下文和查詢來生成答案。
- [create_retrieval_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html) 添加了檢索步驟，並將檢索到的上下文通過鏈傳遞，將其與最終答案一起提供。它的輸入鍵為 `input`，輸出中包含 `input`、`context` 和 `answer`。

In [25]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is a method used by autonomous agents to break down complex tasks into smaller and simpler steps. This process helps the agent plan ahead and understand what needs to be done. It's often achieved through Chain of Thought (CoT) prompting techniques, which instruct the model to "think step by step" and decompose hard tasks into manageable ones.


#### [返回來源​](https://python.langchain.com/v0.2/docs/tutorials/rag/#returning-sources "直接鏈接到返回來源")

在問答應用中，向用戶顯示用來生成答案的來源通常是很重要的。LangChain 的內建函數 `create_retrieval_chain` 會將檢索到的來源文件通過 `"context"` 鍵傳遞到輸出中：

In [26]:
for document in response["context"]:
    print(document)
    print()

page_content='This benchmark evaluates the agent’s tool use capabilities at three levels:

Level-1 evaluates the ability to call the API. Given an API’s description, the model needs to determine whether to call a given API, call it correctly, and respond properly to API returns.
Level-2 examines the ability to retrieve the API. The model needs to search for possible APIs that may solve the user’s requirement and learn how to use them by reading documentation.
Level-3 assesses the ability to plan API beyond retrieve and call. Given unclear user requests (e.g. schedule group meetings, book flight/hotel/restaurant for a trip), the model may have to conduct multiple API calls to solve it.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 21624}

page_content='Fig. 4. Experiments on AlfWorld Env and HotpotQA. Hallucination is a more common failure than inefficient planning in AlfWorld. (Image source: Shinn & Labash, 2023)' metadata={'source': 'https

### [更深入了解​](https://python.langchain.com/v0.2/docs/tutorials/rag/#go-deeper-4 "直接鏈接到更深入了解")

#### [選擇模型](https://python.langchain.com/v0.2/docs/tutorials/rag/#choosing-a-model "直接鏈接到選擇模型")

`ChatModel`：一個由 LLM 支持的聊天模型。接收一系列消息並返回一條消息。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/#chat-models)
-   [整合](https://python.langchain.com/v0.2/docs/integrations/chat/)：提供超過 25 種整合選擇。
-   [介面](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.chat_models.BaseChatModel.html)：基礎介面的 API 參考。

`LLM`：一個文字輸入、文字輸出的 LLM。接收一個字符串並返回一個字符串。

-   [文件](https://python.langchain.com/v0.2/docs/how_to/#llms)
-   [整合](https://python.langchain.com/v0.2/docs/integrations/llms/)：提供超過 75 種整合選擇。
-   [介面](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.llms.BaseLLM.html)：基礎介面的 API 參考。

查看關於本地運行模型的 RAG 指南 [這裡](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)。

#### [自定義提示​](https://python.langchain.com/v0.2/docs/tutorials/rag/#customizing-the-prompt "直接鏈接到自定義提示")

如上所示，我們可以從提示庫中加載提示（例如，[這個 RAG 提示](https://smith.langchain.com/hub/rlm/rag-prompt)）。該提示也可以輕鬆自定義：

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

Check out the [LangSmith trace](https://smith.langchain.com/public/da23c4d8-3b33-47fd-84df-a3a582eedf84/r)

[下一步驟​](https://python.langchain.com/v0.2/docs/tutorials/rag/#next-steps "直接鏈接到下一步驟")  
-----------------------------------------------------------------------------------------------------------

我們已經涵蓋了構建基本問答應用的步驟：

- 使用 [文件載入器](https://python.langchain.com/v0.2/docs/concepts/#document-loaders) 來載入數據
- 使用 [文本分割器](https://python.langchain.com/v0.2/docs/concepts/#text-splitters) 將索引數據進行分塊，使其更容易被模型使用
- [嵌入數據](https://python.langchain.com/v0.2/docs/concepts/#embedding-models) 並將數據存儲在 [向量庫](https://python.langchain.com/v0.2/docs/how_to/vectorstores/) 中
- [檢索](https://python.langchain.com/v0.2/docs/concepts/#retrievers) 先前存儲的數據塊來回應進來的問題
- 使用檢索到的數據塊作為上下文來生成答案

上述每個部分都有許多功能、整合和擴展可以探索。除了上面提到的深入資料外，以下是一些好的下一步驟：

- [返回來源](https://python.langchain.com/v0.2/docs/how_to/qa_sources/): 學習如何返回來源文件
- [串流](https://python.langchain.com/v0.2/docs/how_to/streaming/): 學習如何串流輸出和中間步驟
- [添加聊天歷史](https://python.langchain.com/v0.2/docs/how_to/message_history/): 學習如何將聊天歷史添加到你的應用中
- [檢索概念指南](https://python.langchain.com/v0.2/docs/concepts/#retrieval): 一個高層次的特定檢索技術概述
- [構建本地 RAG 應用](https://python.langchain.com/v0.2/docs/tutorials/local_rag/): 使用所有本地組件創建與上述類似的應用